In [1]:
import pandas as pd
import streamlit as st
from transformers import pipeline
from fuzzywuzzy import process

In [2]:
# Load the dataset
medallists_df = pd.read_csv('Paris_olympics_data/medallists.csv')

In [3]:
medallists_df.head()

,medal_date,medal_type,medal_code,name,gender,country,country_code,nationality,team,team_gender,discipline,event,event_type,url_event,birth_date,code
0,2024-07-27,Gold Medal,1,EVENEPOEL Remco,Male,Belgium,BEL,Belgium,NaN,NaN,Cycling Road,Men's Individual Time Trial,ATH,/en/paris-2024/results/cycling-road/men-s-indi...,2000-01-25,1903136
1,2024-07-27,Silver Medal,2,GANNA Filippo,Male,Italy,ITA,Italy,NaN,NaN,Cycling Road,Men's Individual Time Trial,ATH,/en/paris-2024/results/cycling-road/men-s-indi...,1996-07-25,1923520
2,2024-07-27,Bronze Medal,3,van AERT Wout,Male,Belgium,BEL,Belgium,NaN,NaN,Cycling Road,Men's Individual Time Trial,ATH,/en/paris-2024/results/cycling-road/men-s-indi...,1994-09-15,1903147
3,2024-07-27,Gold Medal,1,BROWN Grace,Female,Australia,AUS,Australia,NaN,NaN,Cycling Road,Women's Individual Time Trial,ATH,/en/paris-2024/results/cycling-road/women-s-in...,1992-07-07,1940173
4,2024-07-27,Silver Medal,2,HENDERSON Anna,Female,Great Britain,GBR,Great Britain,NaN,NaN,Cycling Road,Women's Individual Time Trial,ATH,/en/paris-2024/results/cycling-road/women-s-in...,1998-11-14,1912525


In [4]:
dataset = medallists_df[['medal_date', 'medal_type', 'medal_code', 'name', 'gender', 'country', 'team', 'discipline', 'event']]
dataset.head()

,medal_date,medal_type,medal_code,name,gender,country,team,discipline,event
0,2024-07-27,Gold Medal,1,EVENEPOEL Remco,Male,Belgium,NaN,Cycling Road,Men's Individual Time Trial
1,2024-07-27,Silver Medal,2,GANNA Filippo,Male,Italy,NaN,Cycling Road,Men's Individual Time Trial
2,2024-07-27,Bronze Medal,3,van AERT Wout,Male,Belgium,NaN,Cycling Road,Men's Individual Time Trial
3,2024-07-27,Gold Medal,1,BROWN Grace,Female,Australia,NaN,Cycling Road,Women's Individual Time Trial
4,2024-07-27,Silver Medal,2,HENDERSON Anna,Female,Great Britain,NaN,Cycling Road,Women's Individual Time Trial


In [5]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2271 entries, 0 to 2270
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   medal_date  2271 non-null   object
 1   medal_type  2271 non-null   object
 2   medal_code  2271 non-null   int64 
 3   name        2252 non-null   object
 4   gender      2252 non-null   object
 5   country     2252 non-null   object
 6   team        1502 non-null   object
 7   discipline  2271 non-null   object
 8   event       2271 non-null   object
dtypes: int64(1), object(8)
memory usage: 159.8+ KB


In [6]:
dataset.duplicated().sum()

1

In [7]:
duplicates = dataset[dataset.duplicated()]
# Display duplicated rows
print(duplicates)

     medal_date    medal_type  medal_code name gender country team  \
612  2024-08-09  Bronze Medal           3  NaN    NaN     NaN  NaN   

       discipline                      event  
612  Canoe Sprint  Women's Kayak Double 500m  


In [8]:
dataset.drop_duplicates(inplace=True)

C:\Users\THINKPAD X1 CARBON\AppData\Local\Temp\ipykernel_2984\2870039874.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset.drop_duplicates(inplace=True)


In [9]:
dataset.isna().sum()

medal_date      0
medal_type      0
medal_code      0
name           18
gender         18
country        18
team          768
discipline      0
event           0
dtype: int64

In [10]:
# Fill missing 'team' values with 'Individual'
dataset['team'].fillna('Individual', inplace=True)

C:\Users\THINKPAD X1 CARBON\AppData\Local\Temp\ipykernel_2984\3670833105.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['team'].fillna('Individual', inplace=True)


In [11]:
# Fill 'name', 'gender', 'country' with 'Unknown'
dataset[['name', 'gender', 'country']] = dataset[['name', 'gender', 'country']].fillna('Unknown')

C:\Users\THINKPAD X1 CARBON\AppData\Local\Temp\ipykernel_2984\3985842590.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset[['name', 'gender', 'country']] = dataset[['name', 'gender', 'country']].fillna('Unknown')


In [12]:
dataset.isna().sum()

medal_date    0
medal_type    0
medal_code    0
name          0
gender        0
country       0
team          0
discipline    0
event         0
dtype: int64

In [13]:
dataset.head()

,medal_date,medal_type,medal_code,name,gender,country,team,discipline,event
0,2024-07-27,Gold Medal,1,EVENEPOEL Remco,Male,Belgium,Individual,Cycling Road,Men's Individual Time Trial
1,2024-07-27,Silver Medal,2,GANNA Filippo,Male,Italy,Individual,Cycling Road,Men's Individual Time Trial
2,2024-07-27,Bronze Medal,3,van AERT Wout,Male,Belgium,Individual,Cycling Road,Men's Individual Time Trial
3,2024-07-27,Gold Medal,1,BROWN Grace,Female,Australia,Individual,Cycling Road,Women's Individual Time Trial
4,2024-07-27,Silver Medal,2,HENDERSON Anna,Female,Great Britain,Individual,Cycling Road,Women's Individual Time Trial


In [14]:
# Save the cleaned dataset to a CSV file
dataset.to_csv('cleaned_olympics_data.csv', index=False)

print("Cleaned dataset has been saved as 'cleaned_olympics_data.csv'.")

Cleaned dataset has been saved as 'cleaned_olympics_data.csv'.
